## Capstone

For this project you'll dig into a large amount of text and apply most of what you've covered in this unit and in the course so far.

First, pick a set of texts. This can be either a series of novels, chapters, or articles. Anything you'd like. It just has to have multiple entries of varying characteristics. At least 100 should be good. There should also be at least 10 different authors, but try to keep the texts related (either all on the same topic of from the same branch of literature - something to make classification a bit more difficult than obviously different subjects).

This capstone can be an extension of your NLP challenge if you wish to use the same corpus. If you found problems with that data set that limited your analysis, however, it may be worth using what you learned to choose a new corpus. Reserve 25% of your corpus as a test set.

The first technique is to create a series of clusters. Try several techniques and pick the one you think best represents your data. Make sure there is a narrative and reasoning around why you have chosen the given clusters. Are authors consistently grouped into the same cluster?

Next, perform some unsupervised feature generation and selection using the techniques covered in this unit and elsewhere in the course. Using those features then build models to attempt to classify your texts by author. Try different permutations of unsupervised and supervised techniques to see which combinations have the best performance.

Lastly return to your holdout group. Does your clustering on those members perform as you'd expect? Have your clusters remained stable or changed dramatically? What about your model? Is it's performance consistent?

If there is a divergence in the relative stability of your model and your clusters, delve into why.

Your end result should be a write up of how clustering and modeling compare for classifying your texts. What are the advantages of each? Why would you want to use one over the other? Approximately 3-5 pages is a good length for your write up, and remember to include visuals to help tell your story!

In [188]:
# Grab the data from here: http://share.mailcharts.com/0D0Q2e0L1s47 and http://share.mailcharts.com/0z0F3m1X0l39
import pandas as pd
import numpy as np

In [189]:
df = pd.read_csv("./data/capstone-v2.csv", encoding="ISO-8859-1")
print(df.shape)
print(df.columns)

(3365, 8)
Index(['reg_id', 'add_id', 'name', 'email_guid', 'sent_at', 'subject',
       'full_text', 'r'],
      dtype='object')


In [190]:
df["all_text"] = df["subject"] + " " + df["full_text"].fillna("")

In [243]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import ngrams
import string
import re

# Steps: Clean up text, stemming, remove stop words and weird chars, tokenizer words

# punctuation = list(set(string.punctuation))
re_punctuation = "\.|\>|\/|\)|\"|\(|\}|\'|\_|\-|\$|\:|\[|\^|\+|\?|\`|\~|\!|\<|\@|\;|\=|\*|\\\|\{|\&|\]|\||\,|\|"
stopwords_set = list(set(stopwords.words('english')))
handpicked_works = ["com"]

def get_unigram_sentence(sentence, company_name):
    company_names = company_name.lower().split(" ")
    company_names.append(company_name.lower().replace(" ", ""))
    
    sentence_no_punc = re.sub(re_punctuation, " ", sentence)
    unigram = [word for word in word_tokenize(sentence_no_punc.lower()) if word not in stopwords_set and word not in company_names and word not in handpicked_works]
    return unigram

In [244]:
tokenized_text = []

for i, el in df.iterrows():
    tokenized_text.append(get_unigram_sentence(el['subject'], el['name']))

df["tokenized_text"] = tokenized_text

In [245]:
stemmer = PorterStemmer()

def get_stems(words):
    return [stemmer.stem(word) for word in words]

In [246]:
df["stemmed_tokens"] = df.tokenized_text.apply(lambda x: get_stems(x))

In [247]:
df["stemmed_text"] = df["stemmed_tokens"].apply(lambda x: " ".join(word for word in x))

In [248]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# tfidf = TfidfVectorizer(stop_words="english", min_df=50, ngram_range=(1,2))
# tfidf = tfidf.fit(df["stemmed_text"])
# X = tfidf.transform(df["stemmed_text"])

vectorizer = CountVectorizer(stop_words="english", min_df=50)
vectorizer = vectorizer.fit(df["stemmed_text"])
X = vectorizer.transform(df["stemmed_text"])
X = X.toarray()

In [249]:
# # Split our data in test / train
# from sklearn.model_selection import train_test_split

# X_train, X_test = train_test_split(X, test_size=0.25, random_state=100)
# df_train, df_test = train_test_split(df, test_size=0.25, random_state=100)

# print(X_train.shape)
# print(X_test.shape)

In [250]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

# Note: I had to change this from 6 to 7 to get "ship" back as a category
lda = LDA(6, random_state=22)
# lda.fit(X_train)
lda.fit(X)

/Users/carl/anaconda/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=6, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=22,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [255]:
labels = []

def print_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i]) for i in topic.argsort()[:-top_n - 1:-1]])
        labels.append([(vectorizer.get_feature_names()[i]) for i in topic.argsort()[:-1-1:-1]][0])
        print("=" * 100)

In [256]:
print_topics(lda, vectorizer)

Topic 0:
[('order', 482.15211872257055), ('confirm', 255.62581284201485), ('account', 95.85691550403682), ('receiv', 49.904849740078475), ('lorem', 0.1678661424038993), ('shipment', 0.1673806752796114), ('deliveri', 0.16722944819457455), ('shop', 0.16713224124860648), ('ship', 0.16705559786669114), ('thank', 0.1669234619469451)]
Topic 1:
[('thank', 209.84719430827846), ('order', 155.5153540726883), ('lorem', 104.28456241599727), ('gift', 97.781533536753003), ('way', 87.702285291966646), ('shipment', 53.351322041993669), ('shop', 0.16766427225987832), ('welcom', 0.16721969951005439), ('account', 0.16721608074336233), ('purchas', 0.16712906671594271)]
Topic 2:
[('ship', 314.80376502297815), ('welcom', 299.09929360261737), ('free', 163.94775339994263), ('order', 163.45352267114652), ('today', 98.553129785297514), ('lorem', 0.17229374873845626), ('start', 0.16898496289656215), ('end', 0.16845898481465585), ('10', 0.16768290883576606), ('day', 0.16764003951902143)]
Topic 3:
[('sale', 96.476

In [253]:
# Next, let's assign this to our original text

# X_train_tfidf_lda = lda.transform(X_train)
X_train_tfidf_lda = lda.transform(X)

predicted_label = []

for i in X_train_tfidf_lda:
    # Get the highest value. We'll consider that to be the predicted label.
    predicted_label.append(labels[i.argsort()[-1]])

# df_train["predicted_label"] = predicted_label
df["predicted_label"] = predicted_label

In [254]:
# df_train.head(10)

# Our results are no longer as good
df.subject.head(100)

0                                 Welcome, Lorem Ipsum!
1        Hi! You were looking for free shipping, right?
2                     Le Creuset: New Order # 200068673
3        Your kitchen + our color choices = food heaven
4     Free Shipping Starts Now + Storage Staples to ...
5     Le Creuset: Shipment # 200064834 for Order # 2...
6                                 Make it a Joyful Year
7                Customize your cookware and your inbox
8     Free Shipping Ends Today + Reasons to Love Our...
9              Introducing the Le Creuset Gift Registry
10                        Customer account confirmation
11                        Customer account confirmation
12                              Order #279487 Confirmed
13                              Order #279487 Confirmed
14                    Special Video Message From Cindy!
15                    Special Video Message From Cindy!
16          A shipment from order #279487 is on the way
17          A shipment from order #279487 is on 

In [68]:
# we want to look at "ship" and "order"
# we want to re-classify them using full-text and subject line

# retrain_df = df_train[(df_train["predicted_label"] == "ship") | (df_train["predicted_label"] == "order")]
retrain_df = df[(df["predicted_label"] == "ship") | (df["predicted_label"] == "order")]

In [17]:
# TFIDF

tfidf_retrain = TfidfVectorizer(stop_words="english", min_df=50)
tfidf_retrain = tfidf_retrain.fit(retrain_df["all_text"])
X_retrain = tfidf_retrain.transform(retrain_df["all_text"])
X_retrain = X_retrain.toarray()

In [18]:
# LDA

lda_retrain = LDA(2, random_state = 100)
lda_retrain.fit(X_retrain)

/Users/carl/anaconda/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=2, n_jobs=1,
             n_topics=None, perp_tol=0.1, random_state=100,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [19]:
retrain_labels = []

def print_topics(model, vectorizer, top_n=5):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i]) for i in topic.argsort()[:-top_n - 1:-1]])
        retrain_labels.append([(vectorizer.get_feature_names()[i]) for i in topic.argsort()[:-1-1:-1]][0])
        print("=" * 100)

In [20]:
print_topics(lda_retrain, tfidf_retrain)

Topic 0:
[('shop', 88.784520778743428), ('email', 68.601010910822197), ('xxx', 57.756023962725145), ('free', 53.593783133469266), ('com', 51.285668476048166)]
Topic 1:
[('order', 131.12121518408154), ('shipping', 47.981909790044533), ('lorem', 43.420137896791168), ('shipped', 42.870463573853407), ('00', 39.851922344854316)]


In [21]:
X_retrain_tfidf_lda = lda_retrain.transform(X_retrain)

predicted_label_retrain = []

for i in X_retrain_tfidf_lda:
    # Get the highest value. We'll consider that to be the predicted label.
    predicted_label_retrain.append(retrain_labels[i.argsort()[-1]])

retrain_df["predicted_label_retrain"] = predicted_label_retrain

/Users/carl/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [22]:
# retrain_df[["subject", "predicted_label", "predicted_label_retrain"]]
retrain_df[["subject", "predicted_label", "predicted_label_retrain", "r"]][(retrain_df["predicted_label_retrain"] == "order") & (retrain_df["predicted_label"] == "order")]

# X, order => x
# X, shop => sales

,subject,predicted_label,predicted_label_retrain,r
141,Thank you for your purchase. Order number: A72...,order,order,1
2539,Your order has shipped!,order,order,2
902,Thanks for your order from Bleecker's Finest Deli,order,order,1
250,Your UNIQLO order has shipped! #UNU01202198,order,order,3
973,Your Total Gym order receipt,order,order,2
1567,Thank you! Your order number is 95843113,order,order,3
2638,Your Superior Threads order has shipped,order,order,5
1025,Your Order 3735504 Has Shipped,order,order,9
2560,Your Dillard's Order Status #424067859,order,order,3
1625,Thank you for creating an online account.,order,order,1


In [23]:
# Create complete df with full data (both labels). It's ok to keep cells empty.
# Do crosstab comparison of results v.s. manual labeling
# Question: Can we use R as part of the unsupervised learning
# Use labels from here to do supervised learning



In [74]:
classified = pd.read_csv("./data/captsone-manually-classified.csv")

In [100]:
# Fill blanks with 0
classified.fillna(0, inplace=True)

# Remove any "?" and replace with 0
classified["order confirmation"][classified["order confirmation"] == "?"] = 0

/Users/carl/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [96]:
print(classified["order confirmation"].value_counts())
print(classified["shipping confirmation"].value_counts())
print(classified["delivery notification"].value_counts())

0    1717
1     206
Name: order confirmation, dtype: int64
0.0    1769
1.0     154
Name: shipping confirmation, dtype: int64
0.0    1906
1.0      17
Name: delivery notification, dtype: int64


In [ ]:
# Other models to use aside from LDA: Kmean, nmf
# http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html
# http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
# http://scikit-learn.org/0.15/modules/generated/sklearn.cluster.Ward.html